In [1]:
import hashlib, math, os, subprocess
from multiprocessing import Process
import xlearn as xl
import numpy as np
import pandas as pd
from pandas import DataFrame as DF

In [3]:
data1=pd.read_csv("/home/ubuntu/wangnan/final_data.csv")
data2=pd.read_csv("/data/ng/FFM/mid_tolist.csv")

In [4]:
data1=data1.dropna()
data1['timeStamp']=data1['timeStamp'].apply(lambda x:int(x)-2005)

In [5]:
data3=pd.merge(data1,data2,on='mid')
data3=data3.dropna()

In [6]:
len(data3)

16867572

In [6]:
class FfmEncoder():
    def __init__(self, field_names, label_name, nthread=1):
        self.field_names = field_names
        self.nthread = nthread
        self.label = label_name

    def gen_feats(self, row):
        feats = []
        for field in self.field_names:
            value = row[field]
#             key = field + '-' + str(value)
            key=str(value)
            feats.append(key)
        return feats

    def gen_fm_feats(self, feats):
        feats = ['{0}:{1}:1'.format(field, feat) for (field, feat) in feats]
        return feats

    def convert(self, df, path, i):
        lines_per_thread = math.ceil(float(df.shape[0]) / self.nthread)
        sub_df = df.iloc[i * lines_per_thread: (i + 1) * lines_per_thread]
        tmp_path = path + '_tmp_{0}'.format(i)
        with open(tmp_path, 'w') as f:
            for index,row in sub_df.iterrows():
                feats = []
                for i, feat in enumerate(self.gen_feats(row)):
                    feats.append((i, feat))
                feats = self.gen_fm_feats(feats)
                f.write(str(int(row[self.label])) + ' ' + ' '.join(feats) + '\n')

    def parallel_convert(self, df, path):
        processes = []
        for i in range(self.nthread):
            p = Process(target=self.convert, args=(df, path, i))
            p.start()
            processes.append(p)
        for p in processes:
            p.join()

    def delete(self, path):
        for i in range(self.nthread):
            os.remove(path + '_tmp_{0}'.format(i))

    def cat(self, path):
        if os.path.exists(path):
            os.remove(path)
        for i in range(self.nthread):
            cmd = 'cat {svm}_tmp_{idx} >> {svm}'.format(svm=path, idx=i)
            p = subprocess.Popen(cmd, shell=True)
            p.communicate()

    def transform(self, df, path):
        print('converting data......')
        self.parallel_convert(df, path)
        self.cat(path)
        self.delete(path)

In [9]:
field_names = [i for i in data3.columns if i not in ['star']]

In [10]:
field_names

['uid', 'mid', 'timeStamp', 'timeMode', 'mid_list']

In [11]:
write_path = '/data/ng/FFM/'
fe = FfmEncoder(filed_names,label_name='star',nthread=8)
fe.transform(data3, write_path+'train_without_tag.ffm')

converting data......


In [12]:
ffm_model = xl.create_ffm()
# ffm_model.setTrain("criteo.tr.r100.gbdt0.ffm")
ffm_model.setTrain("/data/ng/FFM/train_without_tag.ffm")
# ffm_model.setValidate("criteo.va.r100.gbdt0.ffm")
# ffm_model.setValidate("val.ffm")

param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':2,           # Size of latent factor
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
#          'Metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }

ffm_model.fit(param, "/data/ng/model2.out")

In [5]:
ffm_model.setTest("val2.ffm")
ffm_model.setSigmoid()
ffm_model.predict("/data/ng/FFM/model2.out", "output2.txt")

In [4]:
ffm_model = xl.create_ffm()